In [13]:
import copy

import cornac
import torch.ao.quantization
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k

from recommenders.utils.timer import Timer

from src.compressors import quantization
from src.metrics import get_model_size, calculate_metrics

In [14]:


MOVIELENS_DATA_SIZE = '100k'

# top k items to recommend
TOP_K = 10

# Model parameters
LATENT_DIM = 100
ENCODER_DIMS = [200]
ACT_FUNC = "tanh"
LIKELIHOOD = "pois"
NUM_EPOCHS = 500
BATCH_SIZE = 1024
LEARNING_RATE = 0.001

In [15]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating"]
)

100%|██████████| 4.81k/4.81k [00:01<00:00, 3.82kKB/s]


In [16]:
train, test = python_random_split(data, 0.75)

In [17]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=1234)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 943
Number of items: 1642


In [18]:
bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=1234,
    # use_gpu=torch.cuda.is_available(),
    use_gpu=False,
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/500 [00:00<?, ?it/s]

Took 101.9498 seconds for training.


In [27]:
base_metrics = calculate_metrics(train,test,bivae)

Took 1.3402 seconds for prediction.
MAP:	0.139413
NDCG:	0.465443
Precision@K:	0.404671
Recall@K:	0.218016


In [20]:
bivae.save('ckpts')

BiVAECF model is saved to chkpts\BiVAECF\2023-04-19_16-02-56-116254.pkl


'chkpts\\BiVAECF\\2023-04-19_16-02-56-116254.pkl'

In [22]:
model_quatized = torch.ao.quantization.quantize_dynamic(
    bivae.bivae,
    {torch.nn.Linear},
    dtype=torch.qint8
)

In [35]:
quantized_bivae = quantization.get_post_dynamic(bivae)
calculate_metrics(train,test,bivae)

Took 1.2100 seconds for prediction.
MAP:	0.139413
NDCG:	0.465443
Precision@K:	0.404671
Recall@K:	0.218016


In [41]:
quantized_bivae.save("ckpts")

BiVAECF model is saved to ckpts\BiVAECF\2023-04-19_16-18-13-155663.pkl


'ckpts\\BiVAECF\\2023-04-19_16-18-13-155663.pkl'

In [39]:
get_model_size(bivae.bivae, "original")

model:  original  	 Size (KB): 2394.711


2394711

In [40]:
get_model_size(model_quatized,"quantized")

model:  quantized  	 Size (KB): 607.695


607695